In [ ]:
import pandas as pd
import sqlite3

In [ ]:
#create a sqlite file
db_name = "us_food_desert.sqlite"

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

In [ ]:
#read our cleaned data csv 
df = pd.read_csv("FoodAccessResearchDataCleanCSV_Updated.csv")

In [ ]:
df.to_sql("food_deserts", conn, if_exists="replace", index=False)

In [ ]:
#query to make sure we imported and created it properly
query = "SELECT * FROM food_deserts LIMIT 5;"
result = pd.read_sql(query, conn)
print(result)

In [ ]:
#inspect column names
print(df.columns)

In [ ]:
#filter certain columns that would apply to states with the worst access to grocery stores
columns_to_filter = ["State", "County_State", "latitude", "longitude", "LowAccess_1MUrban_10MRural", "LowAccessHispLatam_10M", "LowAccessOther_10M", "LowAccessNAAN_10M", "LowAccessNHPI_10M", "LowAccessAsian_10M", "LowAccessBlack_10M"]

In [ ]:
#create a loop for existing columns and the columns we want to filter
existing_columns = [col for col in columns_to_filter if col in df.columns]

In [ ]:
#create a dataframe from the filtered columns
filtered_df = df[existing_columns]
filtered_df.head()

In [ ]:
#remove the null data and create a new cleaned dataframe
no_null_data_df = filtered_df.dropna(how='any')
no_null_data_df.head()

In [ ]:
#look at the max values for the dataframe
columns_to_check = ['LowAccess_1MUrban_10MRural', 'LowAccessHispLatam_10M', 'LowAccessOther_10M', 'LowAccessNAAN_10M', 'LowAccessNHPI_10M', 'LowAccessAsian_10M', 'LowAccessBlack_10M']



max_values = no_null_data_df[columns_to_check].max()
print(max_values)

In [ ]:
#create a dataframe to hold my resulst
results = pd.DataFrame()

# filter the dataframe for california and arizona only
filtered_data = no_null_data_df[no_null_data_df['State'].isin(['California', 'Arizona'])]

#create a for loop to look for the top 5 max values for each column in columns_to_check
for column in columns_to_check:
    top_values = filtered_data.nlargest(5, column)[['State', 'County_State', column]]
    results = pd.concat([results, top_values], ignore_index=True)

print(results)

In [ ]:
#export the results to a csv for further inspection
results.to_csv('top_5_max_values_cali_arizona.csv', index=False, encoding='utf-8')